In [1]:
#機械学習ライブラリ
import numpy as np
import tensorflow as tf
import keras
#モデル作成用
from keras.models import Model
#レイヤー各種
from keras.layers import Dense,GlobalAveragePooling2D
#ジェネレータ
from keras.preprocessing.image import ImageDataGenerator
#オプティマイザ
from keras.optimizers import SGD
#コールバック関数
from keras.callbacks import ModelCheckpoint,CSVLogger,ReduceLROnPlateau
#既存学習済みモデル
from keras.applications.inception_v3 import InceptionV3
#GPUステータス確認用
from tensorflow.python.client import device_lib
#汎用ライブラリ
import os
import cv2

Using TensorFlow backend.


In [5]:
#kerasのセッションクリア
keras .backend.clear_session()
#0番目のGPUを使うように設定
config = tf.ConfigProto(gpu_options = tf.GPUOptions(
            visible_device_list="0",
            allow_growth = True))
session = tf.Session(config = config)
keras.backend.set_session(session = session)
#GPU情報確認
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14360841766109072775, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 1454299545
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 14531255675681491968
 physical_device_desc: "device: 0, name: GeForce GTX 960M, pci bus id: 0000:01:00.0, compute capability: 5.0"]

In [6]:
#画像サイズ
img_size = 299
#チャンネル数
ch = 3
#クラス（カテゴリ）数
class_num = 2
#学習率
lr = 1e-2
#バッチサイズ
batch_size = 8
#エポック数
epochs = 100
#活性化関数
activation = "sigmoid"
#損失関数
loss = "binary_crossentropy"
#カテゴリ数に応じて関数変更
if class_num > 2:
    loss_type = "categorical_crossentropy"
    activation = "softmax"

In [7]:
#既存学習済みモデル読み込み
base_model = InceptionV3(include_top = False,weights = "imagenet")

In [10]:
#特徴量抽出部分を流用する
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024,activation = "relu")(x)
prediction = Dense(class_num,activation = activation)(x)
model = Model(inputs = base_model.input,output = prediction)

D:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  


In [11]:
#248層まで凍結
for layer in base_model.layers[:int(len(base_model.layers) * 0.8)]:
    layer.trainable = layer.name.startswith("batch_normalization")

In [13]:
#オプティマイザ
opt = SGD(lr = lr,momentum = 0.9)
#モデルコンパイル
model.compile(optimizer=opt,loss = loss,etrics=['accuracy'])
#確認
#model.summary()

In [29]:
#画像取得用関数定義

In [30]:
#ジェネレータ定義
traindata_gen = ImageDataGenerator(
                featurewise_center=True,
                featurewise_std_normalization=True,
                rotation_range=20,
                width_shift_range=0.2,
                height_shift_range=0.2,
                horizontal_flip=True)

testdata_gen = ImageDataGenerator()

In [20]:
#学習、テスト用画像取得
#X_train,y_train,X_test,y_test = 
#ジェネレータ作成
#train_generator = traindata_gen()
#test_generator = testdata_gen()

In [ ]:
#コールバック関数定義
#モデル保存先ディレクトリ名
dir_name = "./models/InceptionV3_after_248/"
if not os.path.isdir(dir_name):
    os.mkdir(dir_name)
#良モデル保存用関数
checkpoint = ModelCheckpoint(filepath = dir_name + "model.hdf5",verbose = 1,save_best_only = True)
#ログ取得用関数
csv_logger = CSVLogger(dir_name + "cnn.log")
#lossが減らない場合学習率を減らす用関数
reduce_lr = ReduceLROnPlateau(monitor = "val_los",factor = 0.2,patience = 5,min_lr = 1e-5)

In [22]:
#学習開始
history = model.fit_generator(train_generator,
            steps_per_epoch = int(len(X_train) / batch_size),
            epochs = epochs,
            validation_data = test_generator,
            validation_steps = 100,
            verbose = 1,
            callbacks = [reduce_lr,csv_logger,checkpoint])          

NameError: name 'train_generator' is not defined